In [1]:
source = "CombineSignals";

In [2]:
using NBInclude
import XGBoost
@nbinclude("Alpha.ipynb");

In [3]:
@nbinclude("../TrainingAlphas/XGBoostFeatures.ipynb");

In [4]:
params = read_params(source);

In [5]:
X = zeros(num_items(), length(params["alphas"]))
@showprogress for j = 1:size(X)[2]
    X[:, j] = read_recommendee_alpha(params["alphas"][j]).rating
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [6]:
linreg = X * params["β"];

In [7]:
X = hcat(X, get_xgboost_features("recommendee_inference"));

Progress: 100%|███████████████████████████| Time: 0:00:00 (76.12 ns/it)/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (34.71 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (65.08 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (37.14 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (64.39 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (39.00 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.33 ms/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.15 ms/it)


In [8]:
bst = XGBoost.Booster(model_file = params["bst"]);

In [9]:
predictions = linreg + XGBoost.predict(bst, X);

In [10]:
model(items) = predictions[items];

In [11]:
write_recommendee_alpha(model);